# Combining Dataframes

## Questions

- How do we combine data from multiple sources using pandas?
- How do we add data to an existing dataframe?
- How do we split and combine data columns?

## Objectives

- Use `pd.merge()` to add species info to the survey dataset
- Use `pd.concat()` to add additional rows the dataset
- Use string methods to combine, split, and modify text columns using the `str` accessor

Dataframes can be used to organize and group data by common characteristics. Often, we need to combine elements from separate dataframes into one for analysis and visualization. A merge (or join) allows use to combine two dataframes using values common to each. Likewise, we may need to append data collected under different circumstances. In this chapter, we will show how to merge, concatenate, and split data using pandas.

## Merging dataframes

The survey dataframe we've been using throughout this lesson has a column called species_id. We used this column in the previous lesson to calculate summary statistics about observations of each species. But the species_id is just a two-letter code—what does each code stand for? To find out, we'll now load both the survey dataset and a second dataset containing more detailed information about the various species observed. Read the second dataframe from a file called species.csv:

In [ ]:
import pandas as pd

surveys = pd.read_csv("data/surveys.csv")
species = pd.read_csv("data/species.csv")

species

We can see that the species dataframe includes a genus, species, and taxon for each species_id. This is much more useful than the species_id included in the original dataframe--how can we add that data to our surveys dataframe? Adding it by hand would be tedious and error prone. Fortunately, pandas provides the `pd.merge()` function to join two dataframes.

## Managing repetitive data

Why store species data in a separate table in the first place? Species information is repetitive: Every observation of the same species has the same genus, species, and taxa. Storing it in the original survey table would require including that data in every record, increasing the complexity of the table and creating the possibility of errors. Storing that data in a separate table means we only have to enter and validate it once. A tool like pandas then allows us to access that data when we need it.

To merge the surveys and species dataframes, use:

In [ ]:
merged = pd.merge(surveys, species)
merged

Following the merge, the genus, species, and taxa columns have all been added to the survey dataframe. We can now use those columns to filter and summarize our data.

### Joins

The `pd.merge()` method is equivalent to the JOIN operation in SQL

## Challenge

Filter the merged dataframe to show the genus, the species name, and the weight for every individual captured at the site

In [ ]:
merged[["genus", "species", "weight"]]

In the example above, we didn't provide any information about how we wanted to merge the dataframes together, so pandas used its default arguments to make an educated guess. It looked at the columns in each of the dataframes, then merged them based on the columns that appear in both. Here, the only shared name is species_id, so that's the column pandas used to merge. For more complex tables, we may want to specify the columns are used for merging. We can do so by passing one or more column names using the *on* keyword argument:

In [ ]:
pd.merge(surveys, species, on="species_id")

## Challenge

Compare the number of rows in the original and merged survey dataframes. How do they differ? Why do you think that might be?

**Hint:** Use `pd.unique()` method to look at the species_id column in each dataframe.

In [ ]:
pd.unique(surveys["species_id"].sort_values())

In [ ]:
pd.unique(species["species_id"].sort_values())

Some records in the surveys dataframe do not specify a species. By default, only records with a value that occurs in both the surveys and species dataframes appear in the merged dataframe, so rows without a species_id are excluded.

The built-in `set` type can be used to quickly assess differences like this:

In [ ]:
set(surveys["species_id"]) - set(species["species_id"])

In practice, the values in the columns used to join two dataframes may not align exactly. Above, the surveys dataframe contains a few hundred rows where species_id is NaN. These are the rows that were dropped when the dataframes were merged. 

By default, `pd.merge()` performs an **inner join**. This means that a row will only appear if the value in the shared column appears in both of the datasets being merged. In this case, that means that survey observations that don't have a species_id or have a species_id that does not appear in the species dataframe will be dropped.

This is not always desirable behavior. Fortunately, pandas supports additional types of merges:

- **Inner:** Include all rows with common values in the join column. This is the default behavior.
- **Left:** Include all rows from the left dataframe. Columns from the right dataframe are populated if a common value exists and set to NaN if not.
- **Right:** Include all rows from the right dataframe. Columns from the left dataframe are populated if a common value exists and set to NaN if not.
- **Outer:** Includes all rows from both dataframes

We want to keep all of our observations, so let's do a left join instead. To specify the type of merge, we use the *how* keyword argument:

In [ ]:
pd.merge(surveys, species, how="left")

Now all 35,549 rows appear in the merged dataframe.

## Appending rows to a dataframe

Merges address the case where information about the same set of observations is spread across multiple files. What about when the observations themselves are split into more than one file? For a survey like the one we've been looking at in this lesson, we might get a new file once a year with the same columns but a completely new set of observations. How can we add those new observations to our dataframe?

We'll simulate this operation by splitting data from two different years, 2001 and 2002, into separate dataframes. We can do this using conditionals, as we saw in lesson 3:

In [ ]:
surveys_2001 = surveys[surveys["year"] == 2001].copy()
surveys_2001

In [ ]:
surveys_2002 = surveys[surveys["year"] == 2002].copy()
surveys_2002

We now have two different dataframes with the same columns but different data, one with 1,610 rows, the other with 2,229 rows. We can combine them into a new dataframe using `pd.concat()`, which stacks the dataframes vertically (that is, it appends records from the 2002 dataset to the 2001 dataset). This method accepts a list and will concatenate each item moving from left to right. We're only combining two dataframes here but could do more if needed.

In [ ]:
surveys_2001_2002 = pd.concat([surveys_2001, surveys_2002])
surveys_2001_2002

The combined dataframe includes all rows from both dataframes.

In some cases, the exact columns may change from year to year even within the same project. For example, researchers may decide to add an additional column to track a new piece of data or to provide a quality check. If a column is present in only one dataset, you can still concatenate the datasets. Any column that does not appear in a given dataset will be set to NaN for those rows in the combined dataframe.

To illustrate this, we'll add a validated column to the 2002 survey, then re-run `pd.concat()`:

In [ ]:
surveys_2002["validated"] = True

surveys_2001_2002 = pd.concat([surveys_2001, surveys_2002])
surveys_2001_2002

As expected, the validated column has a value of NaN for the 2001 data in the combined dataframe.

## Joining and splitting columns

Sometimes we'd like to combine values from two or more columns into a single column. For example, we might want to refer to the species in each record by both its genus and species names. In Python, we use the `+` operator to concatenate (or join) strings, and pandas works the same way: 

In [ ]:
species["genus_species"] = species["genus"] + " " + species["species"]
species["genus_species"]

Note that the `+` operator is also used to add numeric columns. In Python, the same operator can be used to perform different operations for different data types (but keep reading for an important caveat.)

Another common need is to join or split dates. In the ecology dataset, the date is split across year, month, and day columns. However, pandas has a special data type, `datetime64`, for representing dates that is useful for plotting, comparing, and resampling time series data. To make use of that functionality, we can concatenate the date columns and convert them to a datetime object. For clarity, we'll use an unambiguous date format: YYYY-MM-DD.

We need to perform an additional step before combining the date columns. Year, month, and day are all stored as integers in our dataframe (specifically, they use the `int64` data type). If we try to concatenate them as they are, we'll receive an error. This is because the `+` operator only works when each object has a similar type, that is, all objects are either the same type or can be coerced to a common type (like `int` and `float`, which are distinct types that generally play well together). In this case, the columns are integers and the hyphens are strings. pandas cannot determine exactly how the user wants to combine the values, so it gives an error.

To resolve the error, we can use the `astype()` method to convert each column to a string before combining the columns:

In [ ]:
year = surveys["year"].astype(str)
month = surveys["month"].astype(str)
day = surveys["day"].astype(str)

surveys["date"] = year + "-" + month + "-" + day
surveys["date"]

Note that some of the dates look a little funny because single-digit days and months do not include a leading zero. For example, in the first row we have **1977-7-16** instead of **1977-07-16**. This is usually not a big deal but can be neatened up using the `str` accessor. 

In pandas, an accessor is an attribute that provides additional functionality to an object. Here, the `str` accessor allows us to access many of the methods from the built-in `str` data type, including `zfill()`, which pads a string to a given length by adding zeroes to the start of the string:

In [ ]:
text = "2"
text.zfill(3)

 Using the `str` accessor, we can use that method to zero-pad the data in a `Series`:

In [ ]:
# Pad month and day to two characters
month = month.str.zfill(2)
day = day.str.zfill(2)

surveys["date"] = year + "-" + month + "-" + day
surveys["date"]

The month and date values in date are now padded to a length of two, allowing us to create a well-formed YYYY-MM-DD date string. Other string methods, like `upper()` and `lower()`, can be used in the same way.

Before we convert the date column to a datetime, we're going to use the date string to show the opposite operation: Splitting a value stored in one column into multiple columns. One way to do this in pandas is to use `str.split()`, which splits each value in a series based on a *delimiter*, a character used as a boundary between parts of a string. Here, a hyphen is used to delimit the year, month, and date in each date. By splitting the column on a hyphen, we can extract each of those components into its own column. We also pass `True` to the *expand* keyword argument, which makes the `str.split()` method return a dataframe:

In [ ]:
surveys["date"].str.split("-", expand=True)

Now let's go ahead and convert our date column into a datetime object using `pd.to_datetime()`:

In [ ]:
surveys["date"] = pd.to_datetime(surveys["date"])
surveys["date"]

## Challenge

pandas can help us ask specific questions which we want to answer about our data. The real skill is to know how to translate our scientific questions into a sensible approach (and subsequently visualize and interpret our results).

Try using pandas to answer the following questions.

1. How many specimens of each sex are there for each year, including those whose sex is unknown?
2. What is the average weight of each taxa?
3. What are the minimum, maximum and average weight for each species of Rodent?
4. What is the average hindfoot length for male and female rodent of each species? Is there a male/female difference?
5. What is the average weight of each rodent species over the course of the years? Is there any noticeable trend for any of the species?

### Show me the solution to challenge 1

How many specimens of each sex are there for each year, including those whose sex is unknown?

In [ ]:
# Fill in NaN values in sex with U
surveys["sex"] = surveys["sex"].fillna("U")

# Count records by sex
result = surveys.groupby(["year", "sex"])["record_id"].count()

# Use the option_context context manager to show all rows
with pd.option_context("display.max_rows", None):
    print(result)

### Show me the solution to challenge 2

What is the average weight of each taxa?

In [ ]:
# Create the merged dataframe
merged = pd.merge(surveys, species, how="left")

# Group by taxa
grouped = merged.groupby("taxa")

# Calculate the min, max, and mean weight
grouped["weight"].mean()

### Show me the solution to challenge 3

What are the minimum, maximum and average weight for each species of Rodent?

In [ ]:
# Create the merged dataframe
merged = pd.merge(surveys, species, how="left")

# Limit merged dataframe to rodents
rodents = merged[merged["taxa"] == "Rodent"]

# Group rodents by species
grouped = rodents.groupby("species_id")

# Calculate the min, max, and mean weight
grouped.agg({"weight": ["min", "max", "mean"]})

### Show me the solution to challenge 4

What is the average hindfoot length for male and female rodent of each species? Is there a male/female difference?

In [ ]:
# Create the merged dataframe
merged = pd.merge(surveys, species, how="left")

# Limit merged dataframe to rodents
rodents = merged[merged["taxa"] == "Rodent"]

# Group rodents by species and sex
grouped = rodents.groupby(["species_id", "sex"])

# Calculate the mean hindfoot length, plus count and standard deviation
# to better assess the question
with pd.option_context("display.max_rows", None):
    print(grouped["hindfoot_length"].agg(["count", "mean", "std"]))

### Show me the solution to challenge 5

What is the average weight of each rodent species over the course of the years? Is there any noticeable trend for any of the species?

In [ ]:
# Create the merged dataframe
merged = pd.merge(surveys, species, how="left")

# Limit merged dataframe to rodents
rodents = merged[merged["taxa"] == "Rodent"]

# Group rodents by species and year
grouped = rodents.groupby(["species_id", "year"])

# Calculate the mean weight by year
result = grouped["weight"].mean()

# Use the option_context context manager to show all rows
with pd.option_context("display.max_rows", None):
    print(result)

## Keypoints

- Combine two dataframes on one or more common values using `pd.merge()`
- Append rows from one dataframe to another using `pd.concat()`
- Combine multiple text columns into one using the `+` operator
- Use the `str` accessor to use string methods like `split()` and `zfill()` on text columns
- Convert date strings to datetime objects using `pd.to_datetime()`